---
title: "Oh, the Places You'll Go! - The Places API"
author: "Spencer Callister"
date: "2024-12-01"
categories: [news]
---


I've mentioned in some of my other posts that I am a big fan of Google Cloud. The APIs are amazing because they are expansive, easy to use, and fairly cost efficient. One of my favorite APIs is the Places API, which gives you access to pretty much everything in Google Maps, including addresses, business details, and reviews. This provides for a really neat data source that you can use to collect a ton of data for your ETL projects.

Follow along to learn how to get a quickstart on using the Places API.

Before you dive in, make sure you have done the following first:
1. [Create a project](https://developers.google.com/workspace/guides/create-project)
2. [Enable billing](https://cloud.google.com/billing/docs/how-to/modify-project#required-permissions-enable) 
3. [enable the Places API](https://console.cloud.google.com/marketplace/product/google/places-backend.googleapis.com?q=search&inv=1&invt=AbkQaA)

Once those steps are completed you can proceed to make a new directory and Python file for your program.

One great use of the Places API is creating recommendation engines. For this tutorial, let's pretend we are writing an application that recommends restaurants based on a user's preferences on the type of food, where they live, price range, etc.

We'll keep this pretty simple by just focusing on getting some of the data that would be helpful for our awesome new app.

To start, import the requests library from Python so you can make the request to the Places endpoint.


In [ ]:
import requests

To access the API, you will need to authenticate by using an API key. To get your API key for the Place API, type "Google Maps Platform" in the Cloud Console search bar and click on the first option.

Hover over the left nav-bar and click on APIs & Services. Then scroll until you find "Places API (New)", make sure it says "New", and click on "Keys".

Select "Show Key" on the Maps Platform API Key and copy the key.

I highly recommend storing your API key in a secret manager like Google Cloud's rather than hard coding it into your program. To learn how to do this, read my previous blog post titled "[Use a Secrets Manager!](https://spencercallister.github.io/how-to-pipeline/posts/secrets_management/)."

When you have the secret ready to go, you can proceed to making the API call.

In my example, I am going to search for Sushi restaurants in Portland, Oregon.

```

key = secret["key"]

url = "https://places.googleapis.com/v1/places:searchText"
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": key,
    "X-Goog-FieldMask": "places.id,places.displayName,places.businessStatus,places.formattedAddress,places.currentOpeningHours,places.currentSecondaryOpeningHours,places.internationalPhoneNumber,places.nationalPhoneNumber,places.priceLevel,places.priceRange,places.rating,places.regularOpeningHours,places.regularSecondaryOpeningHours,places.userRatingCount,places.websiteUri"
}
data = {
    "textQuery": "Popular Sushi in Portland"
}

response = requests.post(url, headers=headers, json=data)

# Print the response
print(response.status_code)

response_data = response.json()
places = response_data["places"]
print(places)

```


The `places` variable should look something like the following when printed out:
```
[{'id': 'ChIJ3X-i8i8LlVQRTW4oCJHmg2M', 'nationalPhoneNumber': '(503) 380-7226'...}...]
```

Looking at really large objects isn't ideal, so let's put the data into a format that is more readable like a data frame.

Since Polars is the future we will be using that.


In [ ]:
import polars as pl

In [ ]:
places = pl.read_json("sample_data/places.json").to_dicts()

Now we can simply plug the places object into the `pl.DataFrame` method and display the data.


In [ ]:
places_df = pl.DataFrame(places)
places_df.limit(5)

Now that we have a lot of the details on the restaurants, we can sort through them based on what the user would like and recommend it back to them.

Since the query is interested in a really popular Sushi place in Portland, I will look for the place with the most reviews.


In [ ]:
rec = places_df\
    .sort("userRatingCount", "rating", descending=[True, True])\
    .limit(1)\
    .to_dicts()[0]

print(f"Based on your preferences, we highly recommend going to {rec['displayName']['text']}, which has an average rating of {rec['rating']} from {rec['userRatingCount']} reviews.")

Let's say the user then wanted to get more details on the restaurant, like whether it has curbside pickup, live music, or alcoholic beverages, you could pass in the ID of the restaurant the user is curious about and get those details for them.

```
url = f"https://places.googleapis.com/v1/places/{places[0]["id"]}"
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": key,
    "X-Goog-FieldMask": "allowsDogs,curbsidePickup,delivery,dineIn,editorialSummary,evChargeOptions,fuelOptions,goodForChildren,goodForGroups,goodForWatchingSports,liveMusic,menuForChildren,parkingOptions,paymentOptions,outdoorSeating,reservable,restroom,reviews,*servesBeer,servesBreakfast,servesBrunch,servesCocktails,servesCoffee,servesDessert,servesDinner,servesLunch,servesVegetarianFood,servesWine,takeout"
}
response = requests.get(url, headers=headers)

# Print the response
print(response.status_code)

response_data = response.json()

```

The places API is full of potential for really great applications, and it is just one of many great APIs that Google has to offer. Thanks for following along and I hope you enjoyed learning about this really awesome tool.